In [1]:
library(tidyverse)
library(brms)
library(here)
library(parallel)
source(here("R/bayesian_helper.R"))
options(mc.cores = 2)
options(warn = 2)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.0.0     ✔ purrr   0.2.5
✔ tibble  1.4.2     ✔ dplyr   0.7.7
✔ tidyr   0.8.1     ✔ stringr 1.3.1
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: Rcpp
Loading 'brms' package (version 2.5.0). Useful instructions
can be found by typing help('brms'). A more detailed introduction
to the package is available through vignette('brms_overview').
Run theme_set(theme_default()) to use the default bayesplot theme.
here() starts at /Users/henrikeckermann/workspace/research_master/minor_research_project/article/analyses/bibo
This is loo version 2.0.0.
**NOTE: As of version 2.0.0 loo defaults to 1 core but we recommend using as many as possible. Use the 'cores' argument or set options(mc.cores = NUM_CORES) for an entire session. Visit mc-sta

ERROR: Error in options(mc.cores = ): argument is missing, with no default


In [2]:
load(here("data/data_transfer.RData"))

In [3]:
df <- data_transfer

In [4]:
# how many models already fitted
m_count <- length(list.files("../models/cc_vs_nocc/skew_normal/")) 

In [13]:
# specify list of genus like 
genus_v <- colnames(df)
genus <- genus_v[-c(1:7)]
# fit already
model_files <- list.files("../models/cc_vs_nocc/skew_normal")
genus_done <- 
    lapply(model_files, function(x) {
        gsub("_time_cc.rds", "", x)
    }) %>% as.character()
# not fit yet
genus_left <- genus[!(genus %in% genus_done)]

In [14]:
genus_left

[1] "Parabacteroides_distasonis_et_rel" "Propionibacterium"                
[3] "Proteus_et_rel"                    "Ruminococcus_obeum_et_rel"        
[5] "Staphylococcus"                    "Sutterella_wadsworthia_et_rel"    
[7] "Tannerella_et_rel"                 "Xanthomonadaceae"                 
[9] "Yersinia_et_rel"

In [15]:
# specify prior 
prior_d <- c(
            set_prior("normal(0, 2)", class = "b"),
            set_prior("exponential(50)", class = "sd"),
            set_prior("normal(0, 10)", class = "Intercept"), 
            set_prior("normal(0, 2)", class = "b", dpar = "sigma"),
            set_prior("exponential(50)", class = "sd", dpar = "sigma"),
            set_prior("normal(0, 5)", class = "Intercept", dpar = "sigma"),
            set_prior("lkj(2)", class = "cor"))


control <-  list(adapt_delta = 0.9999, max_treedepth = 15)
# folder 
folder_d <- here("models/cc_vs_nocc")

# define fitting function 
brmd_sn <- function(genus) {
    # specify formula
    f_d <- as.formula(glue("{genus} ~ 1 + time*cc + age_d_s + (1+ time + age_d_s|subject_id)"))
    formula_d <- bf(f_d, sigma ~ 1 + time*cc + age_d_s + (1+time+age_d_s|subject_id))
    # give individual model name for storage
    model_file <- glue("{folder_d}/skew_normal/{genus}_time_cc")
    #fit model
    brm(
        family = skew_normal(), data = df, formula = formula_d,
        chains = 4, warmup = 1000,
        control = control, prior = prior_d, file = model_file
    )
}

In [ ]:
models <- lapply(genus_left, brmd_sn)

In [ ]:
# help("diagnostic-quantities")
print("hi")

In [33]:
#lapply(genus, function(genus) {
#    ggplot(df, aes_string(genus)) +
#        geom_density()
#})
inspect_d <- 
    mclapply(1:length(genus), function(x) {
        pp_plot_v2(model_p1[[x]], 10, genus_p1[[x]], lower = 0.01, upper = 0)
    })
inspect_d2 <- 
    mclapply(1:length(genus), function(x) {
        pp_plot(model_p1[[x]], 10, genus_p1[[x]], lower = 0.01, upper = 0)
    })

In [ ]:
mclapply(1:length(genus), function(i) {
    print(inspect_d[[i]])
    print(inspect_d2[[i]])
})
